# Digital Marketing Analytics - Group Assignment 1 

### Context

- Multi-channel company with sales of $700M per year
- Retail stores, Catalog, Website
- Brand is well known nationaly, strong positive reputation, good long term customer loyalty
- Food products, purchased as gifts for Christmas

### Marketing challenges

1. CRM (contact management across channels)
2. Competitive marketplace for Christmas gifts
3. Generating business off Christmas

3 differrent marketing channels: __retail, catalogue, website__

## Import libraries

In [52]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

In [53]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

## EDA

### Dataset

4 files:

In [54]:
customers = pd.read_csv('datasets/DMEFExtractSummaryV01.csv')
contacts = pd.read_csv('datasets/DMEFExtractContactsV01.csv')
orders = pd.read_csv('datasets/DMEFExtractOrdersV01.csv')
line_items = pd.read_csv('datasets/DMEFExtractLinesV01.csv')

In [55]:
customers.head()

,Cust_ID,SCF_Code,RetF07Dollars,RetF07Trips,RetF07Lines,RetS07Dollars,RetS07Trips,RetS07Lines,RetF06Dollars,RetF06Trips,RetF06Lines,RetS06Dollars,RetS06Trips,RetS06Lines,RetF05Dollars,RetF05Trips,RetF05Lines,RetS05Dollars,RetS05Trips,RetS05Lines,RetF04Dollars,RetF04Trips,RetF04Lines,RetS04Dollars,RetS04Trips,RetS04Lines,RetPre04Dollars,RetPre04Trips,RetPre04Lines,RetPre04Recency,IntF07GDollars,IntF07NGDollars,IntF07Orders,IntF07Lines,IntS07GDollars,IntS07NGDollars,IntS07Orders,IntS07Lines,IntF06GDollars,IntF06NGDollars,IntF06Orders,IntF06Lines,IntS06GDollars,IntS06NGDollars,IntS06Orders,IntS06Lines,IntF05GDollars,IntF05NGDollars,IntF05Orders,IntF05Lines,IntS05GDollars,IntS05NGDollars,IntS05Orders,IntS05Lines,IntF04GDollars,IntF04NGDollars,IntF04Orders,IntF04Lines,IntS04GDollars,IntS04NGDollars,IntS04Orders,IntS04Lines,IntPre04GDollars,IntPre04NGDollars,IntPre04Orders,IntPre04Lines,IntPre04Recency,CatF07GDollars,CatF07NGDollars,CatF07Orders,CatF07Lines,CatS07GDollars,CatS07NGDollars,CatS07Orders,CatS07Lines,CatF06GDollars,CatF06NGDollars,CatF06Orders,CatF06Lines,CatS06GDollars,CatS06NGDollars,CatS06Orders,CatS06Lines,CatF05GDollars,CatF05NGDollars,CatF05Orders,CatF05Lines,CatS05GDollars,CatS05NGDollars,CatS05Orders,CatS05Lines,CatF04GDollars,CatF04NGDollars,CatF04Orders,CatF04Lines,CatS04GDollars,CatS04NGDollars,CatS04Orders,CatS04Lines,CatPre04GDollars,CatPre04NGDollars,CatPre04Orders,CatPre04Lines,CatPre04Recency,EmailsF07,EmailsS07,EmailsF06,EmailsS06,EmailsF05,EmailsS05,CatCircF07,CatCircS07,CatCircF06,CatCircS06,CatCircF05,CatCircS05,GiftRecF07,GiftRecS07,GiftRecF06,GiftRecS06,GiftRecF05,GiftRecS05,GiftRecF04,GiftRecS04,GiftRecPre04,NewGRF07,NewGRS07,NewGRF06,NewGRS06,NewGRF05,NewGRS05,NewGRF04,NewGRS04,NewGRPre04,FirstYYMM,FirstChannel,FirstDollar,StoreDist,AcqDate,Email,OccupCd,Travel,CurrAff,CurrEv,Wines,FineArts,Exercise,SelfHelp,Collect,Needle,Sewing,DogOwner,CarOwner,Cooking,Pets,Fashion,Camping,Hunting,Boating,AgeCode,IncCode,HomeCode,Child0_2,Child3_5,Child6_11,Child12_16,Child17_18,Dwelling,LengthRes,HomeValue
0,22120,346,0,0,0,0,0,0,14,1,1,0,0,0,0,0,0,0,0,0,70,1,5,0,0,0,0,0,0,99,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,99,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,99,0,0,0,0,0,0,6,1,4,0,8,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,200412,Ret,70,15.93,200412,N,,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,4,8,2,N,N,N,N,N,1,8,0
1,24436,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,99,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,99,0,0,0,0,41,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,99,61,16,37,9,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,200706,Cat,41,,199611,Y,,N,N,N,,N,,,N,N,N,N,N,N,N,N,,N,,,,,N,N,N,N,N,,3,0
2,29278,85,0,0,0,0,0,0,63,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,99,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,99,29,0,1,1,0,0,0,0,27,0,1,1,0,0,0,0,65,0,1,2,0,0,0,0,65,0,1,2,0,0,0,0,25,0,1,1,1,0,0,0,0,0,0,2,0,2,0,3,2,1,0,1,0,2,0,2,0,1,0,0,0,0,0,0,1,0,1,200312,Cat,25,0,200312,N,,Y,N,Y,Y,Y,Y,N,Y,N,N,Y,N,Y,N,Y,N,N,N,7,9,2,N,N,N,N,N,2,2,0
3,50011,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,99,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,36,46,1,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,99,0,0,34,12,42,31,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,200301,Int,82,,199712,Y,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4,51943,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,99,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,24,0,1,1,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,209,0,1,1,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,200112,Cat,209,,199903,N,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


1. "DMEFExtractSummaryV01.csv": __Customer file__.
    - customer ID (*cust_ID*)
    - First 3 digits or the customer's ZIPCode (*SCF_Code*)
    - Purchasing behaviour variables have a naming scheme with 3 components: 
        - Channel: 
            - _Ret_ (retail), 
            - _Int_ (internet)
            - _Cat_ (catalog)
        - Season: _F_ (fall) or _S_ (spring) followed by the last 2 digits of the year. Summaries for the period prior to 2004 have the designation Pre04.
        - Measure:
            - _Dollars_ (Retail sales), 
            - _GDollars_ (Gift), 
            - _NGDollars_ (Non gift),
            -  _Orders_ (number of orders made),
            - _Trips_ (number of trips to retail store)
            -  _Lines_ (Number of different product lines purchased (a product line is a collection of similar products sold under one brand by a single company))
            
            For _Pre04_ (before 2004), _Recency_: recency of last purchase measured in number of season prior to 2004 (99 means not purchased in the channel prior to 2004)

        For example *RetF07Dollars*: customer's Fall 2007 Retail sales dollar total 

    - *EmailsSeasonYY*: Number of mails sent to the customer during this period
    - *CatCircSeasonYY*: Number of catalogs sent to the customer during this period
    - *GiftRecSeasonYY*: "Gifts recipiens counts for this season"
    - *NewGRSeasonYY*: __?__

    - *First YYMM*: Year and Month of the customer's first purchase
    - *FirstChannel*: Channel used for the customer's first purchase 
    - *FirstDollar*: Amount spent on the customer's first purchas
    
    - *AcqDate*: Year and Month that the company added the customer to its database.
    - *StoreDist*: Estimated distance from the customer's address to the nearest store in miles
    - Interest in topics, "Y" or "N" (*Travel, CurrAff, CurrEv, Wines, FineArts, Exercise, SelfHelp, Collect, Needle, Sewing, DogOwner, CarOwner, Cooking, Pets, Fashion, Camping, Hunting* and *Boating*)
    - Presence of children of various age ranges in the household (*Child0_2, Child3_5, Child6_11, Child12_16* and *Child17_18*)
    - _Email_: "Y" or "N", permission to send e-mails to the customer.
    - _HomeValue_: Estimated value of the customer's home in $

    - _AgeCode, IncCode, HomeCode, Dwelling, LengthRes_ and _OccupCd_:  Reference for these codes are not available

In [56]:
contacts.head(5)

,Cust_ID,ContactDate,ContactType
0,22120,20070907,C
1,22120,20071101,C
2,22120,20071011,C
3,22120,20071122,C
4,22120,20071206,C


2. "DMEFExtractContactsV01": __Contact file__.
    - customer ID (*cust_ID*)
    - *ContactDate*: date the contact was made
    - *ContactType*: Catalaog (C) or Email (E)

In [57]:
# Calculate value counts
value_counts = contacts['ContactType'].value_counts()

# Convert to DataFrame for better formatting
value_counts_df = value_counts.reset_index()
value_counts_df.columns = ['Contact Type', 'Count']

# Replace 'E' with 'Email' and 'C' with 'Catalog' in the output DataFrame only
value_counts_df['Contact Type'] = value_counts_df['Contact Type'].replace({'E': 'Email', 'C': 'Catalog'})

# Apply thousand separator
value_counts_df['Count'] = value_counts_df['Count'].apply(lambda x: f"{x:,}")

# Styling the DataFrame for better readability
styled_df = value_counts_df.style.set_table_styles(
    [{
        'selector': 'th',
        'props': [
            ('background-color', '#f4f4f4'),
            ('color', 'black')
        ]
    }]
).set_properties(**{
    'text-align': 'left',
}).hide(axis='index')  # Hide the index column


styled_df

Contact Type,Count
Email,"2,368,315"
Catalog,"1,021,014"


In [58]:
orders.head(5)

,Cust_ID,OrderNum,OrderDate,OrderMethod,PaymentType
0,22120,6642001731,20041213,ST,BC
1,22120,6643008965,20061209,ST,CA
2,24436,7200158663,20070616,P,BC
3,29278,2200159161,20031201,P,BC
4,29278,5800062930,20061208,P,BC


3. "DMEFExtractOrdersV01.csv": __Order file__.
    - customer ID (*cust_ID*)
    - *OrderNum*: order number that links order and line item records
    - *OrderDate*
    - *OrderMethod*: 
        - *ST*: Store
        - *I*: Internet
        - *P*: Phone
        - *M*: Mail
        
        Phone and mail are the two methods by which catalog channel orders are placed

    - *PaymentType*:
        - *BC*: Bankcard
        - *CA*: Cash
        - *CK*: Check
        - *GC*: Gift certificate
        - *HA*: House account
        - *NV* and *PC*: not documented by the company. 
    
        Virtually all payments are bankcard, cash or check.

In [59]:
line_items.head(5)

,Cust_ID,OrderNum,OrderDate,LineDollars,Gift,RecipNum
0,22120,6643008965,20061209,13.95,,
1,22120,6642001731,20041213,10.95,,
2,22120,6642001731,20041213,4.95,,
3,22120,6642001731,20041213,34.95,,
4,22120,6642001731,20041213,10.95,,


4. "DMEFExtractLinesV01.csv": __Order file__.
    - customer ID (*cust_ID*)
    - *OrderNum*: order number that links order and line item records
    - *OrderDate*
    - *LineDollars*: Selling price of the line item in $
    - *Gift*: "Y" or "N", indicates whether the item was given as a gift. Not recorded for Retail channel
    - *RecipNum*: ID number of a gift recipient. Can be used to see if a recipient receives multiple gifts over time.

In [60]:
# Merge the dataframes
df = pd.merge(customers, contacts, on='Cust_ID')
df = pd.merge(df, orders, on='Cust_ID')
df = pd.merge(df, line_items, on='order_ID')

KeyError: 'order_ID'

In [ ]:
customers.columns()

TypeError: 'Index' object is not callable